In [1]:
# Configure environment settings
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Import standard libraries for encoding and mathematical operations
import base64
import io
import math
import pickle

# Import libraries for data manipulation and numerical analysis
import numpy as np
import pandas as pd

# Import I/O libraries for handling byte and string data
from io import BytesIO, StringIO

# Import libraries for web scraping and network operations
import requests
from bs4 import BeautifulSoup
import networkx as nx

# Visualization and interactive tools
import panel as pn
import hvplot.pandas  # Interactive plotting for pandas objects
import hvplot.networkx as hvnx
pn.extension('plotly', sizing_mode="stretch_width")  # Load Plotly support in Panel

# Import libraries for natural language processing and machine learning
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

# OpenAI API client for advanced model interactions
from openai import OpenAI

# Load spaCy's English medium model for NLP tasks
nlp = spacy.load('en_core_web_md')

# Publication Analysis

## Field Popularity

In [2]:
# Load and preprocess dataset
df = pd.read_csv('Derived/All-Journals-Cleaned.csv')
df["Year"] = df['Issue'].str[-4:].astype(int)  # Extract publication year

# Identify columns for authors and JEL codes
author_columns = [col for col in df if col.startswith("Author")]
jel_columns = [col for col in df if col.startswith("JEL")]

# UI widgets for selecting journals and years
journal_names = df['Journal'].unique().tolist()
journal_select = pn.widgets.Select(name='Choose Journal', options=journal_names)

# Journal-specific year range for slider
journal_start_yr = {
    'AMERICAN ECONOMIC REVIEW' : 1999,
    'AMERICAN ECONOMIC REVIEW: INSIGHTS' : 2019,
    'AMERICAN ECONOMIC JOURNAL: APPLIED ECONOMICS' : 2009,
    'AMERICAN ECONOMIC JOURNAL: ECONOMIC POLICY' : 2009,
    'AMERICAN ECONOMIC JOURNAL: MACROECONOMICS' : 2009,
    'AMERICAN ECONOMIC JOURNAL: MICROECONOMICS' : 2009
}
year_slider = pn.widgets.IntSlider(name="Start Year", start=1999, end=2024, step=1)

# Update slider based on journal selection
def update_year_range(event):
    year_slider.start = journal_start_yr[event.new]
    year_slider.value = year_slider.start
journal_select.param.watch(update_year_range, 'value')

# Map JEL codes to subfields
field_code = {
    "A" : "General Economics and Teaching",
    "B" : "History of Economic Thought, Methodology, and Heterodox Approaches",
    "C" : "Mathematical and Quantitative Methods",
    "D" : "Microeconomics",
    "E" : "Macroeconomics and Monetary Economics",
    "F" : "International Economics",
    "G" : "Financial Economics",
    "H" : "Public Economics",
    "I" : "Health, Education, and Welfare",
    "J" : "Labor and Demographic Economics",
    "K" : "Law and Economics",
    "L" : "Industrial Organization",
    "M" : "Business Administration and Business Economics • Marketing • Accounting • Personnel Economics",
    "N" : "Economic History",
    "O" : "Economic Development, Innovation, Technological Change, and Growth",
    "P" : "Political Economy and Comparative Economic Systems",
    "Q" : "Agricultural and Natural Resource Economics • Environmental and Ecological Economics",
    "R" : "Urban, Rural, Regional, Real Estate, and Transportation Economics",
    "Y" : "Miscellaneous Categories",
    "Z" : "Other Special Topics"
} # Dictionary mapping JEL codes to descriptions

# Data transformation for plotting
melted_data_by_JEL = df.melt(id_vars=['Title', 'Year', 'Journal'], value_vars=jel_columns, 
                             var_name='JELCol', value_name='JEL').dropna(subset=['JEL'])
melted_data_by_JEL['Subfield'] = melted_data_by_JEL['JEL'].str[0].map(field_code)
grouped_subfield = melted_data_by_JEL.groupby(['Year', 'Journal', 'Subfield']).size().reset_index(name='counts')
grouped_subfield.columns = ['Year', 'Journal', 'Subfield', 'Count']
igrouped_subfield = grouped_subfield.interactive()

# Interactive pipeline for subfield work counts
subfield_count_pipeline = (
    igrouped_subfield[
        (igrouped_subfield.Year >= year_slider) &
        (igrouped_subfield.Journal == journal_select)
    ].reset_index(drop=True)
)

# Plot setup using hvplot with restricted y and x axes
subfield_trend_plot = subfield_count_pipeline.hvplot(
    x='Year', y='Count', by='Subfield', 
    height=600, width=1050,
    ylim=(0, None), xlim=(None, 2024)
)

# Set up Panel layout with title and plot
subfield_trend_pane = pn.Column(
    "## **Field Popularity**",  # Set bold, header-sized title using Markdown
    pn.Row(subfield_trend_plot)    # Embed plot in a horizontal row
)

## Collaboration Networks

In [3]:
# Initialize collaboration network graph
G = nx.Graph()

# Populate graph with weighted edges based on co-authorships
for index, row in df.iterrows():
    authors = [row[f'{a}'] for a in author_columns if pd.notna(row[f'{a}'])]
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            if G.has_edge(authors[i], authors[j]):
                G[authors[i]][authors[j]]['weight'] += 1  # Update weight for repeated collaborations
            else:
                G.add_edge(authors[i], authors[j], weight=1)  # Add new collaboration edge

# Function to visualize an author's immediate network
def plot_network(author):
    if author in G:
        # Generate and visualize ego graph for selected author
        subgraph = nx.ego_graph(G, author, radius=1)
        positions = nx.spring_layout(subgraph, weight='weight')
        return hvnx.draw(subgraph, positions, edge_color='grey', with_labels=True,
                         label_position='top', font_size='10pt', node_color='darkred', node_line_width=0)
    return hvnx.draw(G.subgraph([]))  # Return an empty graph if no author is selected or found

# Setup author search widget
author_input = pn.widgets.AutocompleteInput(
    name='Search author:', options=list(G.nodes), placeholder='Enter author name'
)

# Bind network plot to author search
network_plot = pn.bind(plot_network, author_input)

# Create panel layout with network search and visualization
network_search_pane = pn.Column(
    "## **Collaboration Networks**",  # Markdown title
    pn.Row(author_input, network_plot)
)

## Theme Discovery

In [4]:
def lemmatize_text(text):
    """
    Convert text to its lemmatized form using spaCy.

    Parameters:
    - text (str): Text to lemmatize.

    Returns:
    - str: Lemmatized text.
    """
    doc = nlp(text)
    return " ".join(token.lemma_ for token in doc)

temp_csv_path = 'Derived/Processed-Journals-Temp.csv'

if not os.path.exists(temp_csv_path):
    # Load dataset and preprocess
    df = pd.read_csv('Derived/All-Journals-Cleaned.csv')
    
    df.dropna(subset=['Abstract'], inplace=True)  # Drop rows with missing Abstract
    df.reset_index(drop=True, inplace=True)  # Reset DataFrame index
    df['Lemmatized_Abstracts'] = df['Abstract'].apply(lemmatize_text)  # Lemmatize Abstracts
    
    df.to_csv(temp_csv_path, index=False)
else:
    df = pd.read_csv('Derived/Processed-Journals-Temp.csv')

In [5]:
# Convert lemmatized documents to list
docs = df['Lemmatized_Abstracts'].tolist()

if os.path.exists('./Derived/embeddings.pkl'):
    # Load embeddings
    with open('./Derived/embeddings.pkl', 'rb') as f:
        embeddings = pickle.load(f)  # Assume it's saved in the current directory
else:
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = sentence_model.encode(docs, show_progress_bar=False)
    # Save embeddings
    with open('./Derived/embeddings.pkl', 'wb') as f:
        pickle.dump(embeddings, f)

In [6]:
# Fit BERTopic model to the documents
if os.path.exists('./Derived/topic-model.pkl'):
    # Load existing topic model
    topic_model = BERTopic.load('./Derived/topic-model.pkl')
else:
    # Fit BERTopic model to the documents
    topic_model = BERTopic().fit(docs, embeddings)
    
    # Update BERTopic with a new CountVectorizer configuration
    vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3), min_df=10)
    topic_model.update_topics(docs, vectorizer_model=vectorizer_model)
    
    # Save the updated topic model
    topic_model.save("./Derived/topic-model.pkl", serialization="pickle")

# Display top topics
top_topics = topic_model.get_topic_info()[1:11]

In [7]:
# Assuming topic_model is your BERTopic instance
fig = topic_model.visualize_topics()
fig.update_layout(
    title='Intertopic Distance Map',
    title_x=0.5,  # Center the title
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2'
)

# Convert the Plotly figure to a Panel pane
theme_pane = pn.Column(
    "## **Theme Discovery**",  # Markdown title
    pn.pane.Plotly(fig),
    sizing_mode='stretch_width'
)

## Build the Sidebar

In [8]:
# Define markdown text for the sidebar
text_current_trends = """
# **Publication Analysis**

**Field Popularity:**

Identify emerging research areas.

**Collaboration Networks:**

Explore patterns of author collaboration.

**Theme Discovery:**

Uncover prevalent research topics.

**Geographical Trends:**

Examine research focuses by country.

**Dataset Insights:**

Reveal frequently used datasets.
"""

# Create a sidebar with markdown content
sidebar_current_trends = pn.layout.WidgetBox(
    pn.pane.Markdown(text_current_trends, margin=(0, 10)),
    max_width=350,
    sizing_mode='stretch_width'
).servable(area='sidebar')

# Tab for Literature Search

## Webscrape JEL Codes

In [9]:
# Set the URL for JEL classification codes
url = 'https://cran.r-project.org/web/classifications/JEL.html'

# Fetch HTML from URL
response = requests.get(url)

# Parse HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find JEL code elements by ID
jel_codes = soup.find_all('li', id=lambda x: x and x.startswith('code:'))

# List to store JEL code descriptions
jel_code_descriptions = []

# Process each JEL code
for code in jel_codes:
    # Split ID to get the JEL code
    jel_code = code['id'].split(':')[1]
    # Check JEL code length
    if len(jel_code) == 3:
        # Get text, remove whitespace
        description = code.get_text(strip=True)
        # Append description to list
        jel_code_descriptions.append(description)

## Clean the Data

In [10]:
# Load dataset from a CSV file
df = pd.read_csv('Derived/All-Journals-Cleaned.csv')

# Convert specified columns to title case
df['Journal'] = df['Journal'].str.title()
df['Issue'] = df['Issue'].str.title()

# Find columns for authors and JEL codes by prefix
author_columns = [col for col in df.columns if col.startswith("Author")]
jel_columns = [col for col in df.columns if col.startswith("JEL")]

# Deduplicate author names, ignoring NaN
unique_authors = pd.unique(df[author_columns].values.ravel('K'))
unique_authors = [author for author in unique_authors if pd.notna(author)]

# Consolidate authors into one column, remove originals
df['Authors'] = df[author_columns].apply(lambda x: '; '.join(x.dropna()), axis=1)
df.drop(columns=author_columns, inplace=True)

# Consolidate JEL codes into one column, remove originals
df['JELs'] = df[jel_columns].apply(lambda x: '; '.join(x.dropna()), axis=1)
df.drop(columns=jel_columns, inplace=True)

# Generate a list of years for potential filtering
years_list = list(range(1999, 2025))

# Extract unique journal names
journal_list = df['Journal'].unique().tolist()

# Ensure the 'Abstract' column is treated as strings
df['Abstract'] = df['Abstract'].astype(str)

# Create a new column called 'Similarity Score' with the value 'NA'
df['Similarity_Score'] = None

# Function to convert URLs to clickable links
def make_clickable(url, name):
    return f'<a target="_blank" href="{url}">{name}</a>'

# Apply clickable link function to 'Link' column
df['Link'] = df.apply(lambda row: make_clickable(row['Link'], row['Link']), axis=1)

## Build the Sidebar

In [11]:
# Define interactive widgets for data filtering
year_input = pn.widgets.MultiChoice(name='Year(s)', options=years_list)  # Selection widget for years
journal_input = pn.widgets.MultiChoice(name='Journal(s)', options=journal_list)  # Selection widget for journals
author_input = pn.widgets.MultiChoice(name='Author(s)', options=unique_authors)  # Selection widget for authors
multi_choice = pn.widgets.MultiChoice(name='JEL Code(s)', options=jel_code_descriptions)  # Selection widget for JEL codes
abstract_search = pn.widgets.TextInput(name='Search Keyword')  # Text input for keyword search

# Initialize interactive widgets for text input
text_area_input = pn.widgets.TextAreaInput(
    name='Research Interests',
    placeholder='Enter some sentences about what interests you, and we will help recommend articles.',
    height=400
)

def get_filtered_data():
    """
    Generate and return CSV format of the filtered DataFrame.
    
    Returns:
        StringIO: Represents a CSV formatted string of filtered data.
    """
    # Return CSV data if DataFrame is not empty
    if global_filtered_df is not None:
        return StringIO(global_filtered_df.to_csv(index=False))
    # Return headers only if DataFrame is empty
    return StringIO("Title,Issue,Journal,Abstract,Authors,Link\n")

# Setup file download for the filtered data
download_button = pn.widgets.FileDownload(
    callback=get_filtered_data,
    filename="filtered_data.csv",
    button_type="primary"
)

# Markdown text detailing literature search functionality
text_lit_search = """
# **Literature Search**

Search literature by selecting criteria such as years, journals, JEL codes, authors, and keywords.

**Real-Time Updates:**

Updates display dynamically based on selected filters.

**Articles Recommendation:**

Enter text to measure similarity with the displayed articles.

**Download Data:**

Download filtered literature data in CSV format.
"""

# Additional explanation text about the search functionality
explanation_lit_search = """
For more information on the JEL Classification System, see [this website](https://www.aeaweb.org/econlit/jelCodes.php?view=jel).
"""

# Setup sidebar with widgets and explanatory texts
sidebar_lit_search = pn.layout.WidgetBox(
    pn.pane.Markdown(text_lit_search, margin=(0, 10)),
    year_input,
    journal_input,
    multi_choice,
    author_input,
    abstract_search,
    text_area_input,
    pn.pane.Markdown(explanation_lit_search, margin=(10, 0)),
    download_button,
    max_width=350,
    sizing_mode='stretch_width'
).servable(area='sidebar')

## Build the Main Panel

In [12]:
# Variable to store the latest filtered DataFrame
global_filtered_df = None

def filter_data(selected_years, selected_journals, selected_jel_options, 
                selected_authors, abstract_query, detailed_interests):
    """
    Filters the DataFrame based on selected widget criteria.

    Args:
        selected_years (list): List of selected years.
        selected_journals (list): List of selected journals.
        selected_jel_options (list): List of selected JEL codes.
        selected_authors (list): List of selected authors.
        abstract_query (str): Input string for abstract search.

    Returns:
        DataFrame: Filtered DataFrame according to selected filters.
    """
    global global_filtered_df  # Reference global variable to store the filtered results
    filtered_df = df  # Start with the full dataset for filtering
    
    # Apply filters based on user selection from widgets
    if selected_years:
        year_strings = [str(year) for year in selected_years]
        filtered_df = filtered_df[filtered_df['Issue'].apply(lambda issue: any(year in issue for year in year_strings))]
        
    if selected_journals:
        filtered_df = filtered_df[filtered_df['Journal'].isin(selected_journals)]
    
    if selected_authors:
        filtered_df = filtered_df[filtered_df['Authors'].apply(lambda authors: all(author in authors for author in selected_authors))]

    if selected_jel_options:
        cleaned_jel_options = [option.split(':')[0] for option in selected_jel_options]
        filtered_df = filtered_df[filtered_df['JELs'].apply(lambda x: all(jel_option in x for jel_option in cleaned_jel_options))]
        
    if abstract_query:
        filtered_df = filtered_df[
            filtered_df['Abstract'].str.contains(abstract_query, case=False, na=False) |
            filtered_df['Title'].str.contains(abstract_query, case=False, na=False) |
            filtered_df['Issue'].str.contains(abstract_query, case=False, na=False) |
            filtered_df['Journal'].str.contains(abstract_query, case=False, na=False) |
            filtered_df['Authors'].str.contains(abstract_query, case=False, na=False)
        ]
    
    
    if detailed_interests:
        # Convert input text to embeddings
        doc = nlp(detailed_interests)  
        embeddings = doc.vector

        # Calculate cosine similarity with abstracts and sort the results
        filtered_df['Similarity_Score'] = filtered_df['Abstract'].apply(
            lambda x: cosine_similarity([nlp(x).vector], [embeddings])[0][0]
        )
        filtered_df = filtered_df.sort_values(by='Similarity_Score', ascending=False)

    global_filtered_df = filtered_df[['Title', 'Issue', 'Journal', 'Abstract', 'Authors', 'Link', 'Similarity_Score']]
 
    # Stylize the DataFrame
    filtered_df = global_filtered_df.style.set_properties(**{'text-align': 'left'}).hide(axis="index")
    
    return filtered_df

# Binds filter function with widgets to update display dynamically
dynamic_view = pn.bind(
    filter_data,
    selected_years=year_input.param.value,
    selected_journals=journal_input.param.value,
    selected_jel_options=multi_choice.param.value,
    selected_authors=author_input.param.value,
    abstract_query=abstract_search.param.value,
    detailed_interests = text_area_input.param.value
)

# Tab for Literature Review

## Upload a File

In [13]:
# Initialize the file input widget and other components
file_input = pn.widgets.FileInput(sizing_mode='stretch_width')
global_df = None
output_pane = pn.pane.Markdown()
df_output = pn.pane.DataFrame()
explanations_pane = pn.pane.Markdown(sizing_mode='stretch_width')

def check_file_format_and_columns(event):
    global global_df
    filename = file_input.filename
    desired_columns = {'Title', 'Abstract'}
    
    # Check if the uploaded file is in CSV format
    if filename.endswith('.csv'):
        # Convert the uploaded file content into a DataFrame
        df = pd.read_csv(io.BytesIO(file_input.value))
        # Identify any missing required columns
        missing_columns = desired_columns - set(df.columns)
        
        # If no columns are missing, proceed with displaying and updating data
        if not missing_columns:
            # If the DataFrame has 50 or fewer records, use all of them
            if len(df) <= 50:
                output_pane.object = "Congratulations! Your input information is shown on the right."
                df = df[['Title', 'Abstract']][:len(df)]  # Simplified slicing for clarity
            else:
                # If the DataFrame has more than 50 records, use only the first 50
                output_pane.object = "Only the first 50 papers will be used. Your input information is shown on the right."
                df = df[['Title', 'Abstract']][:50]
            # Update global DataFrame and output pane with filtered data
            df_output.object = df.style.set_properties(**{'text-align': 'left'}).hide(axis="index")
            global_df = df
        else:
            # Inform the user about any missing required columns
            output_pane.object = f"Missing columns: {', '.join(missing_columns)}."
            global_df = None
            explanations_pane.object = ''
    else:
        # Alert the user if the uploaded file is not a CSV
        output_pane.object = "Incorrect format, please upload a .csv file."
        global_df = None
        explanations_pane.object = ''

# Watch for changes in the value of the file input and trigger the checking function
file_input.param.watch(check_file_format_and_columns, 'value')

Watcher(inst=FileInput(sizing_mode='stretch_width'), cls=<class 'panel.widgets.input.FileInput'>, fn=<function check_file_format_and_columns at 0x3357f2020>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

## Build the Main Panel

In [14]:
# Function to update explanations based on the global dataframe
def update_explanations():
    global explanations_pane, global_df

    # Check if there is a valid DataFrame available
    if global_df is not None:
        # Ensure 'Title' and 'Abstract' columns are treated as strings
        global_df['Title'] = global_df['Title'].astype(str)
        global_df['Abstract'] = global_df['Abstract'].astype(str)

        # Concatenate 'Title' and 'Abstract' for each record
        concatenated_strings = 'Title: ' + global_df['Title'] + ' \nAbstract: ' + global_df['Abstract']
        concatenated_list = concatenated_strings.tolist()
        input_string = '\n\n'.join(concatenated_list)

        # Calculate the number of observations and suggested number of groups
        num_obs = len(global_df)
        num_group = max(1, int(math.sqrt(num_obs)))  # Ensure at least one group

        # Initialize OpenAI client with API key
        client = OpenAI(api_key=os.getenv('OPEN_AI_KEY'))

        # Create a chat completion with OpenAI to generate explanations
        response = client.chat.completions.create(
            model='gpt-3.5-turbo-0125',
            messages=[
                {
                    'role': 'system',
                    'content': 'You are an economics research assistant who is good at conducting literature reviews.'
                },
                {
                    'role': 'user',
                    'content': f'''
                    Based on the provided list of papers {input_string}, categorize them into distinct {num_group} groups based on topics, datasets, or methodological similarities. Then, provide concise explanations for each group, mentioning the titles of the papers. Aim to avoid redundancy and be as succinct as possible.

                    I need all the papers to be categorized into groups. I want {num_group} groups in total. For each group, follow the format:

                    **Group: Impact of Pollution on Health and Productivity**

                    **Titles:**
                    
                    - The Mortality and Medical Costs of Air Pollution: Evidence from Changes in Wind Direction
                    - The Impact of Pollution on Worker Productivity
                    - A Simple Auction Mechanism for the Optimal Allocation of the Commons
                    - ....
                    - ....
                    - ....
                    
                    **Explanation:**
                    
                    This group focuses on the implications of pollution on health and productivity. The first paper examines the causal effects of pollution exposure on mortality and healthcare costs. The second paper investigates the impact of pollution on worker productivity in service sectors. The third paper proposes an auction mechanism for efficiently allocating pollution permits in a regulatory framework.
                    
                    In the end, please have a **Difference** section to contrast the groups you categorized.
                    '''
                }
            ]
        )

        # Extract and display the explanations from the response
        explanations = response.choices[0].message.content
        explanations_pane.object = explanations
    else:
        # Display a message when no valid data is available for generating explanations
        explanations_pane.object = 'No valid data available for generating explanations.'

# Initialize the update button with an event handler for updating explanations
update_button = pn.widgets.Button(name='Update Categorization', button_type='primary')
update_button.on_click(lambda event: update_explanations())

Watcher(inst=Button(button_type='primary', name='Update Categorization', sizing_mode='stretch_width'), cls=<class 'panel.widgets.button.Button'>, fn=<function <lambda> at 0x34704c360>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

## Build the Sidebar

In [15]:
# Text for the AEA Literature Review Tab
text_lit_review = """
# **Literature Review**

**Upload Dataset:**

Upload your dataset in CSV format containing paper titles and abstracts.

**Update Categorization:**

After uploading, press the button to categorize the literature based on similarity.

Please ensure your dataset contains the necessary columns: 'Title' and 'Abstract'.
"""

# Explanation text for the AEA Literature Review section
explanation_lit_review = """
Consider using the Literature Search Section to generate the .csv file. Users can also upload papers from other journals they are interested in. This tab uses the model 'gpt-3.5-turbo-0125'.
"""

# Sidebar for the AEA Literature Review section
sidebar_lit_review = pn.layout.WidgetBox(
    pn.pane.Markdown(text_lit_review, margin=(0, 10)),
    file_input,
    output_pane,
    pn.pane.Markdown(explanation_lit_review, margin=(10, 0)),
    update_button,
    max_width=350,
    sizing_mode='stretch_width'
).servable(area='sidebar')

# Template for the Website

In [16]:
# Logo file path
file_path = 'logo.png'
mime_type = "image/png"

# Open the logo image file in binary read mode
with open(file_path, "rb") as img_file:
    # Read the file content and encode it in Base64
    encoded_string = base64.b64encode(img_file.read()).decode('utf-8')
    
# Create a data URI for the encoded image
data_uri = f"data:{mime_type};base64,{encoded_string}"

# Define Markdown content for README tab
tab0_content = pn.pane.Markdown("README", sizing_mode='stretch_width')

# Rows for displaying different analysis and search panes
row1 = pn.Row(subfield_trend_pane)  # Subfield trends
row2 = pn.Row(network_search_pane)  # Network search
row3 = pn.Row(theme_pane)           # Theme discovery

# Column layout for main tab, includes dividers for visual separation
tab_main = pn.Column(
    row1,
    pn.layout.Divider(),
    row2,
    pn.layout.Divider(),
    row3,
    sizing_mode='stretch_width'
)

# Rows that combine sidebars with main content for each tab
tab1 = pn.Row(sidebar_current_trends, tab_main)  # Publication Analysis
tab2 = pn.Row(sidebar_lit_search, dynamic_view)  # Literature Search
tab3 = pn.Row(sidebar_lit_review, df_output, explanations_pane)  # Literature Review

# Tabs for navigating between different sections of the application
tabs = pn.Tabs(
    ("README", tab0_content),
    ("Publication Analysis", tab1),
    ("Literature Search", tab2),
    ("Literature Review Support", tab3),
    sizing_mode='stretch_width'
)

# Main application layout using FastListTemplate
layout = pn.template.FastListTemplate(
    title='American Economic Association Publication Dynamics and Research Support',
    logo=data_uri,  # Application logo
    theme_toggle=False,  # Theme switching disabled
    main=pn.Column(tabs),
    accent='#622433'  # Accent color
)

# Serve the application layout
layout.show()

Launching server at http://localhost:49652
